In [1]:
import os
import sys
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd

import functools

import numpy as np
import math
import random
import scipy

import plotting
import utils
import yaml

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from data_loader import BiosDataset, JigsawBias, CustomDataset, CrowSPairs, StereoSet, TwitterAAE, SBICDataset, ImplicitHate, WinoQueer

2025-11-04 16:26:48.691979: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762270008.707347 2650593 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762270008.712886 2650593 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762270008.727017 2650593 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762270008.727034 2650593 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762270008.727036 2650593 computation_placer.cc:177] computation placer alr

In [2]:
CONFIG_FILE = 'configs/esann25/experiment_config.json'

with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)

openai_models = config["openai_models"]
huggingface_models = config["huggingface_models"]
models = huggingface_models + openai_models
emb_dir = '../%s' % config['embedding_dir']

In [3]:
def get_dataset_stats(g_train, g_test, groups):
    stats = {'train': {group: 0 for group in groups}, 'test': {group: 0 for group in groups}}
    df = pd.DataFrame(stats)
    
    if g_test.ndim == 1:
        # single-label
        if len(g_train) > 0:
            df.loc[groups[1], 'train'] = np.sum(g_train)
            df.loc[groups[0], 'train'] = len(g_train)-np.sum(g_train)
        df.loc[groups[1], 'test'] = np.sum(g_test)
        df.loc[groups[0], 'test'] = len(g_test)-np.sum(g_test)
        
        df.loc['total', :] = df.loc[groups, :].sum(axis=0)
    else:
        # multi-label
        for i, group in enumerate(groups):
            if len(g_train) > 0:
                df.loc[group, 'train'] = np.sum(g_train[:,i])
            df.loc[group, 'test'] = np.sum(g_test[:,i])
        df.loc['total', 'train'] = len(g_train)
        df.loc['total', 'test'] = len(g_test)
    return df


def print_dataset_stats(ds: CustomDataset, as_percentage=True):
    # protected groups
    group_dist = {split: {group: 0 for group in ds.group_names} for split in ds.splits}
    group_df = pd.DataFrame(group_dist)

    for split in ds.splits:
        for i, group in enumerate(ds.group_names):
            group_df.loc[group, split] = np.sum(ds.protected_groups[split][:,i])
        total = ds.n_samples[split]
        if as_percentage:
            group_df.loc[:,split] *= 100/total
        group_df.loc['total #', split] = total

    if as_percentage:
        group_df.columns = [(col+' [%]') for col in group_df.columns]
    
    print("protected groups:")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        print(group_df)
    print()

    # classes
    if not ds.unlabeled:
        lbl_dist = {split: {cls: 0 for cls in ds.class_names} for split in ds.splits}
        lbl_df = pd.DataFrame(lbl_dist)
    
        for split in ds.splits:
            if ds.multi_label:
                for i, cls in enumerate(ds.class_names):
                    lbl_df.loc[cls, split] = np.sum(ds.labels[split][:,i])
            else: # single-label
                for i, cls in enumerate(ds.class_names):
                    lbl_df.loc[cls, split] = np.sum(ds.labels[split] == i)
            total = ds.n_samples[split]
            if as_percentage:
                lbl_df.loc[:,split] *= 100/total
            lbl_df.loc['total #', split] = total

        if as_percentage:
            lbl_df.columns = [(col+' [%]') for col in lbl_df.columns]
    
        print("class label:")
        print(lbl_df)

In [4]:
winoqueer = WinoQueer(local_dir='../../../data/winoqueer_final.csv')
print_dataset_stats(winoqueer)

load winoqueer
compute class weights for split test
protected groups:
                  test [%]
Transgender       4.576197
Bisexual          6.640316
Lesbian           2.127800
Asexual           3.943786
NB                1.901625
LGBTQ            12.700922
Queer             9.486166
Pansexual         2.687747
Gay               5.935441
Heterosexual     16.214317
Cis               8.785683
Cisgender         8.785683
Straight         16.214317
total #       91080.000000

class label:
           test [%]
queer          50.0
normative      50.0
total #     91080.0


/tmp/ipykernel_2650423/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 4.57619675  6.64031621  2.12779974  3.94378568  1.90162495 12.70092227
  9.48616601  2.68774704  5.93544137 16.21431708  8.78568292  8.78568292
 16.21431708]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total


In [5]:
implicithate = ImplicitHate(local_dir='../../../data/implicit-hate-corpus/')
print_dataset_stats(implicithate)

Loading Implicit Hate dataset with option all from: ../../../data/implicit-hate-corpus/
compute class weights for split test
protected groups:
                    test [%]
female              0.251397
male                0.484171
homosexual          0.293296
lgbtq+              0.344507
latino/latina       0.260708
middle eastern      0.256052
asian               0.395717
non-white           2.081006
white               5.381750
black               3.114525
progressive         0.432961
conservative        0.442272
liberal             0.665736
illegal             0.702980
immigrant           4.515829
minorities          3.342644
jewish              2.397579
muslim/islam        3.170391
religion            5.758845
total #         21480.000000

class label:
                     test [%]
implicit_hate       33.054004
explicit_hate        5.069832
not_hate            61.876164
white_grievance      7.174115
irony                3.756983
other                0.367784
stereotypical        5.2

/tmp/ipykernel_2650593/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.25139665 0.48417132 0.29329609 0.34450652 0.26070764 0.25605214
 0.39571695 2.08100559 5.38175047 3.11452514 0.43296089 0.44227188
 0.66573557 0.70297952 4.51582868 3.34264432 2.39757914 3.17039106
 5.75884544]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_2650593/1241408030.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[33.05400372  5.0698324  61.87616387  7.17411546  3.75698324  0.36778399
  5.20018622  5.82402235  4.25512104  3.12849162]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [4]:
implicithate = ImplicitHate(local_dir='../../../data/implicit-hate-corpus/', option='positive-only')
print_dataset_stats(implicithate)

Loading Implicit Hate dataset with option positive-only from: ../../../data/implicit-hate-corpus/
compute class weights for split test
protected groups:
                   test [%]
female             0.659421
male               1.269996
homosexual         0.769325
lgbtq+             0.903651
latino/latina      0.683844
middle eastern     0.671633
asian              1.037978
non-white          5.458542
white             14.104286
black              8.169496
progressive        1.135670
conservative       1.160093
liberal            1.746245
illegal            1.843937
immigrant         11.845158
minorities         8.767859
jewish             6.288924
muslim/islam       8.316034
religion          15.105630
total #         8189.000000

class label:
                    test [%]
implicit_hate      86.701673
explicit_hate      13.298327
white_grievance    18.805715
irony               9.854683
other               0.964709
stereotypical      13.640249
incitement         15.276591
inferiority  

/tmp/ipykernel_2650593/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.65942117  1.26999634  0.7693247   0.90365124  0.68384418  0.67163268
  1.03797778  5.45854195 14.10428624  8.16949566  1.1356698   1.16009281
  1.74624496  1.84393699 11.84515814  8.76785932  6.28892417  8.3160337
 15.1056295 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_2650593/1241408030.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[86.70167298 13.29832702 18.80571498  9.85468311  0.96470876 13.64024911
 15.27659055 11.16131396  8.20613017]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [7]:
stereoset = StereoSet()
print_dataset_stats(stereoset)

load Stereoset with option both
Load inter- and intrasentence samples and merge them to one dataset
compute class weights for split val
protected groups:
                       val [%]
european             10.254591
white                 1.024671
african              13.872468
black                 1.229605
hispanic              2.601088
latin-american        2.411918
hispanic/latino       5.005123
south east asian      5.359817
middle east          13.210373
male                 26.026641
female               15.448885
christian             1.505478
muslim                1.891700
hindu                 1.166548
jewish                0.173406
total #           12687.000000

class label:
                      val [%]
anti-stereotype     33.333333
stereotype          33.333333
unrelated           33.333333
total #          12687.000000


/tmp/ipykernel_2650423/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[10.25459131  1.02467092 13.87246788  1.22960511  2.60108773  2.41191771
  5.00512335  5.35981714 13.21037282 26.02664144 15.44888469  1.50547805
  1.89170017  1.16654844  0.17340585]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_2650423/1241408030.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[33.33333333 33.33333333 33.33333333]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [8]:
csp = CrowSPairs()
print_dataset_stats(csp)

load crowspairs
compute class weights for split test
protected groups:
                                       test [%]
black                                 10.444297
white                                 12.665782
asian                                  3.514589
poor                                   4.244032
rich                                   2.751989
male                                  43.103448
female                                23.309019
physical disability                    1.094164
mental disability                      0.629973
physical illness                       0.961538
mental illness                         0.464191
foreigner, immigrant, undocumented     0.530504
canadian                               0.397878
us american                            5.935013
mexican                                1.591512
chinese                                0.961538
italian                                0.928382
british                                0.464191
russian          

/tmp/ipykernel_2650423/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[10.44429708 12.66578249  3.51458886  4.24403183  2.75198939 43.10344828
 23.30901857  1.09416446  0.62997347  0.96153846  0.46419098  0.53050398
  0.39787798  5.93501326  1.59151194  0.96153846  0.92838196  0.46419098
  0.29840849  0.62997347  2.05570292  1.92307692  0.62997347  1.42572944
  0.82891247  0.96153846  1.06100796  0.59681698  2.81830239  3.28249337
  3.01724138  1.29310345  1.55835544]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_2650423/1241408030.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[50. 50.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [9]:
taae = TwitterAAE()
print_dataset_stats(taae)

load twitterAAE


README.md:   0%|          | 0.00/274 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


test-00000-of-00001.parquet:   0%|          | 0.00/4.15M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100000 [00:00<?, ? examples/s]

compute class weights for split test
protected groups:
         test [%]
aa           50.0
white        50.0
total #  100000.0



In [40]:
bios = BiosDataset(option='unsupervised')
print_dataset_stats(bios)

load BIOS with option unsupervised
protected groups:
             train [%]      test [%]       dev [%]
male         53.899751     53.898798     53.897382
female       46.100249     46.101202     46.102618
total #  257478.000000  99069.000000  39642.000000

class label:
                       train [%]      test [%]       dev [%]
accountant              1.421481      1.422241      1.422733
architect               2.550898      2.551757      2.552848
attorney                8.221673      8.219524      8.218556
chiropractor            0.669960      0.670240      0.671005
comedian                0.708410      0.709606      0.711367
composer                1.412548      1.414166      1.415166
dentist                 3.681480      3.681273      3.680440
dietitian               0.996978      0.998294      0.998941
dj                      0.374401      0.375496      0.378387
filmmaker               1.765199      1.766446      1.768327
interior_designer       0.368575      0.369439      0.3682

/tmp/ipykernel_122704/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[53.89975066 46.10024934]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_122704/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.42148067  2.55089755  8.22167331  0.66996015  0.70841004  1.41254787
  3.68147958  0.99697838  0.37440092  1.76519936  0.36857518  5.03343975
  1.89025859  4.78332129  1.95162305  0.44508657  0.63617086  0.36041914
  6.12596028 10.3496221   1.77024833 29.80759521  4.63923131  0.35381664
  1.74461507  3.42903083  4.09005818  0.41789978]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [7]:
local_dir = '../../../data/bios_huggingface_merge.pkl'
bios = BiosDataset(option='supervised', local_dir=local_dir)
print_dataset_stats(bios)

load BIOS with option supervised
compute class weights for split train
compute class weights for split test
compute class weights for split dev
protected groups:
           train [%]  test [%]      dev [%]
male       57.346444     56.12    57.839388
female     42.653556     43.88    42.160612
total #  7017.000000   2500.00  1046.000000

class label:
                train [%]  test [%]      dev [%]
architect        3.648283      3.40     3.154876
surgeon         10.089782     10.16    10.038241
dentist          1.667379      1.12     1.720841
teacher         12.042183     13.08    12.141491
psychologist     9.633747      8.68     9.177820
nurse            6.156477      6.32     5.736138
photographer    17.030070     16.72    17.304015
physician       11.742910     10.80    10.420650
attorney        13.025509     14.48    15.105163
journalist      17.300841     18.04    17.304015
total #       7017.000000   2500.00  1046.000000


/tmp/ipykernel_2650593/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[57.34644435 42.65355565]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_2650593/1241408030.py:60: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 3.64828274 10.08978196  1.66737922 12.04218327  9.63374662  6.15647713
 17.03006983 11.74291008 13.02550948 17.30084082]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [5]:
local_dir = '../../../data/jigsaw_bias'
jigsaw = JigsawBias(option='multi-class', local_dir=local_dir)
print_dataset_stats(jigsaw)

load JigsawBias with option: multi-class
protected groups:
                                         train [%]       dev [%]      test [%]
female                                   11.159822     10.230500     10.080623
male                                      8.363369      8.040201      7.380553
transgender                               0.392480      0.426043      0.384178
white                                     3.797435      3.561285      3.181646
asian                                     0.674060      0.704610      0.589795
black                                     2.069832      1.878960      1.834316
latino                                    0.245191      0.245794      0.297603
atheist                                   0.265407      0.507975      0.611439
buddhist                                  0.101947      0.071007      0.097397
christian                                 7.610755      7.324667      7.375142
hindu                                     0.102235      0.081931      0.

/tmp/ipykernel_123613/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.11598221e+01 8.36336857e+00 3.92479640e-01 3.79743545e+00
 6.74059955e-01 2.06983192e+00 2.45191475e-01 2.65407497e-01
 1.01946514e-01 7.61075492e+00 1.02235315e-01 1.48327846e+00
 3.58949922e+00 1.61728181e-01 2.71472304e-02 1.56731936e+00
 4.62080518e-03 4.04320453e-03 7.16802403e-01]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_123613/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2.92641368 0.15190897 0.38930284 2.55328366 0.06382487 0.06498007]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [4]:
local_dir = '../../../data/jigsaw_bias'
jigsaw = JigsawBias(option='single-class', local_dir=local_dir)
print_dataset_stats(jigsaw)

load JigsawBias with option: single-class
protected groups:
                                         train [%]       dev [%]      test [%]
female                                   11.312843     10.346472     10.161748
male                                      8.495626      8.195592      7.488709
transgender                               0.413703      0.455605      0.393866
white                                     4.237590      4.111040      3.523790
asian                                     0.678955      0.704598      0.577670
black                                     2.380265      2.230345      2.153135
latino                                    0.256289      0.254291      0.309841
atheist                                   0.274775      0.519178      0.614431
buddhist                                  0.103356      0.074168      0.099779
christian                                 7.649733      7.347955      7.420439
hindu                                     0.102516      0.084764      0

/tmp/ipykernel_130334/2816108568.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.13128433e+01 8.49562628e+00 4.13703472e-01 4.23758960e+00
 6.78955462e-01 2.38026548e+00 2.56288881e-01 2.74775292e-01
 1.03355844e-01 7.64973293e+00 1.02515552e-01 1.53941387e+00
 3.86478031e+00 1.77861682e-01 2.82898109e-02 1.78393867e+00
 4.48155420e-03 3.92135993e-03 7.20970032e-01]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  group_df.loc[:,split] *= 100/total
/tmp/ipykernel_130334/2816108568.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[94.81680247  5.18319753]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  lbl_df.loc[:,split] *= 100/total


In [6]:
local_dir='../../../data/filtered_sbic_minority_overview.csv'
sbic = SBICDataset(local_dir=local_dir)
print_dataset_stats(sbic)

protected groups:
                                       train [%]     test [%]      dev [%]
white                                   0.333955     0.361318     0.277778
black                                   8.312693    14.410202    13.611111
asian                                   1.775527     2.146652     2.371795
non-white                               8.479670    14.495218    13.717949
latin-american                          0.116884     0.233794     0.235043
hispanic                                0.976818     1.530287     1.239316
mixed race                              0.016698     0.021254     0.064103
middle eastern                          0.484235     0.488842     0.555556
indigenous                              0.467537     0.701382     0.619658
race                                    9.495450    15.642933    14.850427
male                                    1.730999     1.806589     2.029915
female                                  8.802494     9.691817     9.401709
non-bin

/tmp/ipykernel_464509/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3.33954860e-01 8.31269307e+00 1.77552667e+00 8.47967050e+00
 1.16884201e-01 9.76817967e-01 1.66977430e-02 4.84234548e-01
 4.67536805e-01 9.49544987e+00 1.73099936e+00 8.80249353e+00
 2.78295717e-02 4.34141319e-01 1.80892216e+00 2.47683188e-01
 1.49166504e+00 4.34141319e-01 2.76904238e+00 1.45270364e+00
 4.62249186e+00 4.14660618e-01 7.15219993e-01 9.85166838e-01
 7.06871121e-01 1.94807002e-02 3.42303732e-01 3.84048089e-01
 4.11877661e-01 1.25233073e-01 1.30798987e-01 2.22636574e-01
 9.07244038e-01 2.67163888e-01 7.18002950e-01 1.89241088e-01
 1.94807002e-02 7.23568864e-02 4.92583419e+00 9.54554309e-01
 3.42303732e-01 3.89614004e-01 8.82197423e-01 9.21158823e-01
 1.28016030e-01 3.22823032e-01 1.69760387e-01 5.53808477e-01
 4.31358361e-01 9.74035010e-02 4.17443576e-02 9.46205438e-02
 2.78295717e-02 7.23568864e-02 6.95739293e-02 3.3

In [6]:
local_dir='../../../data/filtered_sbic_minority_overview.csv'
sbic = SBICDataset(local_dir=local_dir, option='offensive')
print_dataset_stats(sbic)

load SBIC with local file: ../../../data/filtered_sbic_minority_overview.csv
compute class weights for split train
compute class weights for split test
compute class weights for split dev
protected groups:
                                       train [%]     test [%]      dev [%]
white                                   0.333955     0.508373     0.277778
black                                   8.312693    20.275120    13.611111
asian                                   1.775527     3.020335     2.371795
non-white                               8.479670    20.394737    13.717949
latin-american                          0.116884     0.328947     0.235043
hispanic                                0.976818     2.153110     1.239316
mixed race                              0.016698     0.029904     0.064103
middle eastern                          0.484235     0.687799     0.555556
indigenous                              0.467537     0.986842     0.619658
race                                    9.49

/tmp/ipykernel_2650593/1241408030.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[3.33954860e-01 8.31269307e+00 1.77552667e+00 8.47967050e+00
 1.16884201e-01 9.76817967e-01 1.66977430e-02 4.84234548e-01
 4.67536805e-01 9.49544987e+00 1.73099936e+00 8.80249353e+00
 2.78295717e-02 4.34141319e-01 1.80892216e+00 2.47683188e-01
 1.49166504e+00 4.34141319e-01 2.76904238e+00 1.45270364e+00
 4.62249186e+00 4.14660618e-01 7.15219993e-01 9.85166838e-01
 7.06871121e-01 1.94807002e-02 1.45548660e+00 3.42303732e-01
 3.84048089e-01 4.11877661e-01 1.25233073e-01 1.30798987e-01
 4.25792447e-01 2.22636574e-01 9.07244038e-01 2.67163888e-01
 7.18002950e-01 1.89241088e-01 1.94807002e-02 7.23568864e-02
 4.92583419e+00 9.54554309e-01 3.42303732e-01 3.89614004e-01
 8.82197423e-01 9.21158823e-01 1.28016030e-01 3.22823032e-01
 1.69760387e-01 5.53808477e-01 4.31358361e-01 9.74035010e-02
 4.17443576e-02 9.46205438e-02 2.78295717e-02 7.